In [1]:
import turicreate as tc
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import spdiags
from scipy.stats import multivariate_normal
from copy import deepcopy
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

In [2]:
wiki = tc.SFrame('people_wiki.gl/').head(5000)

In [3]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

tf_idf = load_sparse_csr('4_tf_idf.npz')  # NOT people_wiki_tf_idf.npz
map_index_to_word = tc.SFrame('4_map_index_to_word.gl/')  # NOT people_wiki_map_index_to_word.gl

In [4]:
tf_idf = normalize(tf_idf)

In [5]:
def diag(array):
    n = len(array)
    return spdiags(array, 0, n, n)

def logpdf_diagonal_gaussian(x, mean, cov):
    '''
    Compute logpdf of a multivariate Gaussian distribution with diagonal covariance at a given point x.
    A multivariate Gaussian distribution with a diagonal covariance is equivalent
    to a collection of independent Gaussian random variables.

    x should be a sparse matrix. The logpdf will be computed for each row of x.
    mean and cov should be given as 1D numpy arrays
    mean[i] : mean of i-th variable
    cov[i] : variance of i-th variable'''

    n = x.shape[0]
    dim = x.shape[1]
    assert(dim == len(mean) and dim == len(cov))

    # multiply each i-th column of x by (1/(2*sigma_i)), where sigma_i is sqrt of variance of i-th variable.
    scaled_x = x.dot( diag(1./(2*np.sqrt(cov))) )
    # multiply each i-th entry of mean by (1/(2*sigma_i))
    scaled_mean = mean/(2*np.sqrt(cov))

    # sum of pairwise squared Eulidean distances gives SUM[(x_i - mean_i)^2/(2*sigma_i^2)]
    return -np.sum(np.log(np.sqrt(2*np.pi*cov))) - pairwise_distances(scaled_x, [scaled_mean], 'euclidean').flatten()**2

In [6]:
def log_sum_exp(x, axis):
    '''Compute the log of a sum of exponentials'''
    x_max = np.max(x, axis=axis)
    if axis == 1:
        return x_max + np.log( np.sum(np.exp(x-x_max[:,np.newaxis]), axis=1) )
    else:
        return x_max + np.log( np.sum(np.exp(x-x_max), axis=0) )

def EM_for_high_dimension(data, means, covs, weights, cov_smoothing=1e-5, maxiter=int(1e3), thresh=1e-4, verbose=False):
    # cov_smoothing: specifies the default variance assigned to absent features in a cluster.
    #                If we were to assign zero variances to absent features, we would be overconfient,
    #                as we hastily conclude that those featurese would NEVER appear in the cluster.
    #                We'd like to leave a little bit of possibility for absent features to show up later.
    n = data.shape[0]
    dim = data.shape[1]
    mu = deepcopy(means)
    Sigma = deepcopy(covs)
    K = len(mu)
    weights = np.array(weights)

    ll = None
    ll_trace = []

    for i in range(maxiter):
        # E-step: compute responsibilities
        logresp = np.zeros((n,K))
        for k in range(K):
            logresp[:,k] = np.log(weights[k]) + logpdf_diagonal_gaussian(data, mu[k], Sigma[k])
        ll_new = np.sum(log_sum_exp(logresp, axis=1))
        if verbose:
            print(ll_new)
        logresp -= np.vstack(log_sum_exp(logresp, axis=1))
        resp = np.exp(logresp)
        counts = np.sum(resp, axis=0)

        # M-step: update weights, means, covariances
        weights = counts / np.sum(counts)
        for k in range(K):
            mu[k] = (diag(resp[:,k]).dot(data)).sum(axis=0)/counts[k]
            mu[k] = mu[k].A1

            Sigma[k] = diag(resp[:,k]).dot( data.multiply(data)-2*data.dot(diag(mu[k])) ).sum(axis=0) \
                       + (mu[k]**2)*counts[k]
            Sigma[k] = Sigma[k].A1 / counts[k] + cov_smoothing*np.ones(dim)

        # check for convergence in log-likelihood
        ll_trace.append(ll_new)
        if ll is not None and (ll_new-ll) < thresh and ll_new > -np.inf:
            ll = ll_new
            break
        else:
            ll = ll_new

    out = {'weights':weights,'means':mu,'covs':Sigma,'loglik':ll_trace,'resp':resp}

    return out

In [7]:
from sklearn.cluster import KMeans

np.random.seed(5)
num_clusters = 25

# Use scikit-learn's k-means to simplify workflow
kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=-1)
kmeans_model.fit(tf_idf)
centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_

means = [centroid for centroid in centroids]

/home/lui/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


In [8]:
num_docs = tf_idf.shape[0]
weights = []
for i in range(num_clusters):
    # Compute the number of data points assigned to cluster i:
    num_assigned = len(cluster_assignment[cluster_assignment == i])
    if num_assigned == 0:
        num_assigned = 1
    w = float(num_assigned) / num_docs
    weights.append(w)#Initializing covariances. To initialize our covariance parameters, we compute $\hat{\sigma}_{k, j}^2 = \sum_{i=1}^{N}(x_{i,j} - \hat{\mu}_{k, j})^2$ for each feature $j$.  For features with really tiny variances, we assign 1e-8 instead to prevent numerical instability. We do this computation in a vectorized fashion in the following code block.

In [9]:
covs = []
for i in range(num_clusters):
    member_rows = tf_idf[cluster_assignment==i]
    cov = (member_rows.multiply(member_rows) - 2*member_rows.dot(diag(means[i]))).sum(axis=0).A1 / member_rows.shape[0] \
          + means[i]**2
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)

In [10]:
out = EM_for_high_dimension(tf_idf, means, covs, weights, cov_smoothing=1e-10)
print(out['loglik']) # print history of log-likelihood over time

[3879297479.366981, 4883345753.533131, 4883345753.533131]


In [11]:
for i in range(num_clusters):
    indices = out['means'][i].argsort()[-5:][::-1]
    values = [out['means'][i][indice] for indice in indices]
    print(indices)
    print(values)
    break

[ 99044  97954  98817 100028  99227]
[0.1514622106302948, 0.06333877620223939, 0.059097721078932755, 0.04767219464090771, 0.04676920111312222]


In [12]:
# Fill in the blanks
def visualize_EM_clusters(tf_idf, means, covs, map_index_to_word):
    print('')
    print('==========================================================')

    num_clusters = len(means)
    for c in range(num_clusters):
        print('Cluster {0:d}: Largest mean parameters in cluster '.format(c))
        print('\n{0: <12}{1: <12}{2: <12}'.format('Word', 'Mean', 'Variance'))
        
        # The k'th element of sorted_word_ids should be the index of the word 
        # that has the k'th-largest value in the cluster mean. Hint: Use np.argsort().
        sorted_word_ids = means[c].argsort()[-5:][::-1]

        for i in sorted_word_ids[:5]:
            print('{0: <12}{1:<10.2e}{2:10.2e}'.format(map_index_to_word['category'][i], 
                                                       means[c][i],
                                                       covs[c][i]))
        print('\n=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====')

In [13]:
'''By EM'''
visualize_EM_clusters(tf_idf, out['means'], out['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
poetry      1.51e-01    1.90e-02
poems       6.33e-02    6.45e-03
poet        5.91e-02    6.36e-03
de          4.77e-02    8.72e-03
literary    4.68e-02    3.29e-03

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
she         1.60e-01    4.59e-03
her         1.04e-01    3.20e-03
music       1.53e-02    1.04e-03
actress     1.52e-02    1.14e-03
show        1.27e-02    7.33e-04

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
football    7.45e-02    4.57e-03
club        5.84e-02    2.55e-03
league      5.72e-02    2.

minister    1.28e-01    7.34e-03
prime       4.89e-02    4.79e-03
government  3.65e-02    2.00e-03
party       3.32e-02    1.33e-03
cabinet     3.25e-02    3.34e-03

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 21: Largest mean parameters in cluster 

Word        Mean        Variance    
news        5.64e-02    6.42e-03
radio       4.04e-02    3.61e-03
show        3.08e-02    2.28e-03
television  2.40e-02    1.07e-03
reporter    2.38e-02    1.88e-03

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 22: Largest mean parameters in cluster 

Word        Mean        Variance    
theatre     5.66e-02    7.14e-03
film        4.28e-02    1.66e-03
actor       3.92e-02    3.01e-03
television  3.67e-02    1.74e-03
comedy      3.52e-02    4.57e-03

==========

In [14]:
np.random.seed(5)
num_clusters = len(means)
num_docs, num_words = tf_idf.shape

random_means = []
random_covs = []
random_weights = []

for k in range(num_clusters):
    
    # Create a numpy array of length num_words with random normally distributed values.
    # Use the standard univariate normal distribution (mean 0, variance 1).
    # YOUR CODE HERE
    mean = np.random.normal(0, 1, num_words)
    
    # Create a numpy array of length num_words with random values uniformly distributed between 1 and 5.
    # YOUR CODE HERE
    cov = np.random.uniform(1, 5, num_words)

    # Initially give each cluster equal weight.
    # YOUR CODE HERE
    weight = [1/num_clusters]
    
    random_means.append(mean)
    random_covs.append(cov)
    random_weights.append(weight)

In [15]:
out_random_init = EM_for_high_dimension(tf_idf, random_means, random_covs, random_weights, cov_smoothing=1e-5)
print(out_random_init['loglik'][-1]) # print history of log-likelihood over time

2362875609.1670547


In [16]:
print('Smaller')

Smaller


In [17]:
'''By EM'''
visualize_EM_clusters(tf_idf, out_random_init['means'], out_random_init['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
she         4.21e-02    5.79e-03
her         2.63e-02    1.82e-03
music       2.03e-02    2.37e-03
singapore   1.80e-02    5.72e-03
bbc         1.20e-02    1.82e-03

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
he          1.38e-02    1.11e-04
she         1.29e-02    1.67e-03
university  1.06e-02    3.19e-04
music       1.05e-02    9.94e-04
league      1.04e-02    9.58e-04

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
she         3.30e-02    3.96e-03
her         2.43e-02    2.57e-03
music       1.46e-02    1.

she         2.37e-02    4.14e-03
health      1.34e-02    3.64e-03
her         1.16e-02    9.68e-04
he          1.15e-02    9.46e-05
australian  1.13e-02    1.90e-03

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 21: Largest mean parameters in cluster 

Word        Mean        Variance    
he          1.35e-02    9.19e-05
university  1.22e-02    3.46e-04
she         1.19e-02    1.71e-03
football    9.61e-03    1.29e-03
his         9.58e-03    7.85e-05

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 22: Largest mean parameters in cluster 

Word        Mean        Variance    
church      1.71e-02    3.31e-03
she         1.51e-02    2.41e-03
he          1.39e-02    1.05e-04
her         1.19e-02    1.47e-03
music       1.10e-02    1.27e-03

==========

In [18]:
print('Less interpretable')

Less interpretable
